## Data pre-processing

In [25]:
# Import necessary libraries
import pandas as pd
import numpy as np

In [26]:
df = pd.read_csv("../data/interim/nhanes_data_interim.csv")

### Convert categorical variables represented by float to int

In [27]:
df["diabetes"] = df["diabetes"].astype(int)
df["current_smoking_freq"] = df["current_smoking_freq"].astype(int)

### Transform highly-skewed features

In [28]:
# Log transformation on right-skewed features

cols_to_transform = ["hba1c_percentage", "fpg_mmol_L", "hdl_mmol_L"]

for col in cols_to_transform:
    df[col + "_log"] = np.log(df[col])

# Drop original columns

df = df.drop(cols_to_transform, axis=1)

### Identify outliers

In [29]:
def three_sd_range(series):
    mean = series.mean()
    sd = series.std()
    lower_bound = mean - 3 * sd
    upper_bound = mean + 3 * sd
    
    return (lower_bound, upper_bound)

In [30]:
for col_name in ["bmi", "age", "hba1c_percentage_log", "fpg_mmol_L_log", "hdl_mmol_L_log"]:
    lower_bound, upper_bound = three_sd_range(df[col_name])
    outlier_present = (df[col_name].min() < lower_bound or df[col_name].max() > upper_bound)
    print(f"{col_name} has at least one outlier: {outlier_present}")
    
    if outlier_present:
        curr_length = df[col_name].count()
        df = df[(df[col_name] >= lower_bound) & (df[col_name] <= upper_bound)]
        new_length = df[col_name].count()
        print(f"{curr_length - new_length} row has been removed")
    
    print()

bmi has at least one outlier: True
11 row has been removed

age has at least one outlier: False

hba1c_percentage_log has at least one outlier: True
31 row has been removed

fpg_mmol_L_log has at least one outlier: True
27 row has been removed

hdl_mmol_L_log has at least one outlier: True
6 row has been removed



In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1134 entries, 0 to 1208
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   bmi                       1134 non-null   float64
 1   high_blood_pressure       1134 non-null   int64  
 2   gender                    1134 non-null   int64  
 3   age                       1134 non-null   int64  
 4   diabetes                  1134 non-null   int32  
 5   lifetime_100_cigs_smoked  1134 non-null   int64  
 6   current_smoking_freq      1134 non-null   int32  
 7   hba1c_percentage_log      1134 non-null   float64
 8   fpg_mmol_L_log            1134 non-null   float64
 9   hdl_mmol_L_log            1134 non-null   float64
dtypes: float64(4), int32(2), int64(4)
memory usage: 88.6 KB


In [32]:
# Save the pre-processed DataFrame to a new CSV file
df.to_csv("../data/processed/nhanes_data_processed.csv", index=False)

### Categorical encoding

In [33]:
# Map multiclass values to more readable strings to prepare for one-hot encoding
df["diabetes"] = df["diabetes"].map({1: "yes", 2: "no", 3: "borderline"})
df["current_smoking_freq"] = df["current_smoking_freq"].map({1: "every_day", 2: "some_days", 3: "none"})

In [34]:
# One-hot encode the multiclass columns "diabetes" and "current_smoking_frequency"
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False, dtype=int)
one_hot_encoded = encoder.fit_transform(df[["diabetes", "current_smoking_freq"]])

encoded_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(["diabetes", "current_smoking_freq"]))

# Concatenate the original DataFrame with the one-hot encoded DataFrame
df = pd.concat([df, encoded_df], axis=1)

# Drop the original multiclass columns after encoding
df = df.drop(columns=["diabetes", "current_smoking_freq"], axis=1)

In [35]:
# Resultant DataFrame after one-hot encoding
df.sample(5)

,bmi,high_blood_pressure,gender,age,lifetime_100_cigs_smoked,hba1c_percentage_log,fpg_mmol_L_log,hdl_mmol_L_log,diabetes_borderline,diabetes_no,diabetes_yes,current_smoking_freq_every_day,current_smoking_freq_none,current_smoking_freq_some_days
532,31.2,0.0,1.0,78.0,1.0,1.774952,1.791759,-0.072571,0.0,1.0,0.0,1.0,0.0,0.0
322,26.4,0.0,1.0,40.0,1.0,1.667707,1.724551,0.405465,0.0,1.0,0.0,1.0,0.0,0.0
662,22.8,1.0,0.0,42.0,1.0,1.722767,1.490654,0.727549,0.0,1.0,0.0,0.0,1.0,0.0
769,24.2,0.0,1.0,55.0,1.0,1.410987,1.724551,0.239017,0.0,1.0,0.0,0.0,1.0,0.0
129,23.4,1.0,1.0,72.0,1.0,1.722767,1.733424,0.371564,0.0,1.0,0.0,0.0,1.0,0.0


In [36]:
# Save the one-hot encoded pre-processed DataFrame to a new CSV file
df.to_csv("../data/processed/nhanes_data_processed_one_hot.csv", index=False)